# Preliminary Results

## Load Libraries

In [7]:
library(tidyverse)
library(repr)
library(digest)
library(infer)
library(gridExtra)
suppressWarnings(library(cowplot))

## Data Intake and Pre-Processing

<mark>TODO</mark>: insert some text to explain what we are doing

In [8]:
data <- read.csv(file = "data/athlete_events.csv")

In [9]:
processed_df <- data[order(data$Year),]  %>% 
    filter(!is.na(Age)) # get rid of NA/Age
    # we should get rid of anyone who didn't win a medal either - does NA in the medal mean they didn't win or they don't know
    # also, let's decide if we care about bronze/silver as opposed to just looking at gold

first_time_only <- processed_df[match(unique(processed_df$ID), processed_df$ID),] # only first time winners now, if they won a second time, it was excluded
# I think this works, someone check though
    
winter_df <- first_time_only %>% 
    filter(Season == "Winter")

summer_df <- first_time_only %>% 
    filter(Season == "Summer")

head(first_time_only)
head(winter_df)
head(summer_df)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,12068,Arthur Charles Blake,M,24,NA,NA,United States,USA,1896 Summer,1896,Summer,Athina,Athletics,"Athletics Men's 1,500 metres",Silver
3,12563,Conrad Helmut Fritz Bcker,M,25,NA,NA,Germany,GER,1896 Summer,1896,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NA
10,12929,John Mary Pius Boland,M,25,NA,NA,Great Britain,GBR,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Singles,Gold
12,16616,"Thomas Edmund ""Tom"" Burke",M,21,183,66,United States,USA,1896 Summer,1896,Summer,Athina,Athletics,Athletics Men's 100 metres,Gold
14,17492,Eugne Henri Callot,M,20,NA,NA,France,FRA,1896 Summer,1896,Summer,Athina,Fencing,"Fencing Men's Foil, Individual",Silver
15,18785,Demetrius Emmanuel Casdagli,M,23,NA,NA,Greece,GRE,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Singles,Silver


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,391,Clarence John Abel,M,23,185,102,United States,USA,1924 Winter,1924,Winter,Chamonix,Ice Hockey,Ice Hockey Men's Ice Hockey,Silver
2,992,Josef Adolf,M,25,NA,NA,Czechoslovakia,TCH,1924 Winter,1924,Winter,Chamonix,Nordic Combined,Nordic Combined Men's Individual,NA
3,1077,Xavier Affentranger,M,26,NA,NA,Switzerland,SUI,1924 Winter,1924,Winter,Chamonix,Ski Jumping,"Ski Jumping Men's Normal Hill, Individual",NA
4,1341,Johan Petter hln (Andersson-),M,44,NA,NA,Sweden,SWE,1924 Winter,1924,Winter,Chamonix,Curling,Curling Men's Curling,Silver
5,2329,Louis Albert,M,25,NA,NA,France,FRA,1924 Winter,1924,Winter,Chamonix,Ski Jumping,"Ski Jumping Men's Normal Hill, Individual",NA
6,2431,Henri Eugne Aldebert,M,43,NA,NA,France-1,FRA,1924 Winter,1924,Winter,Chamonix,Bobsleigh,Bobsleigh Men's Four/Five,NA


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,12068,Arthur Charles Blake,M,24,NA,NA,United States,USA,1896 Summer,1896,Summer,Athina,Athletics,"Athletics Men's 1,500 metres",Silver
2,12563,Conrad Helmut Fritz Bcker,M,25,NA,NA,Germany,GER,1896 Summer,1896,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NA
3,12929,John Mary Pius Boland,M,25,NA,NA,Great Britain,GBR,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Singles,Gold
4,16616,"Thomas Edmund ""Tom"" Burke",M,21,183,66,United States,USA,1896 Summer,1896,Summer,Athina,Athletics,Athletics Men's 100 metres,Gold
5,17492,Eugne Henri Callot,M,20,NA,NA,France,FRA,1896 Summer,1896,Summer,Athina,Fencing,"Fencing Men's Foil, Individual",Silver
6,18785,Demetrius Emmanuel Casdagli,M,23,NA,NA,Greece,GRE,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Singles,Silver
